In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pose
from pose_parser import parse_file, detect_perspective


In [2]:
good_videos = [parse_file("dataset/bicep/bicep_good_"+ str(i) +".npy") for i in range(1,7)]
bad_videos = [parse_file("dataset/bicep/bicep_bad_" + str(i) + ".npy") for i in range(1,11)]



Data shape:  (114, 18, 3)
Mean torso:  360.11485467535044
Data shape:  (101, 18, 3)
Mean torso:  361.91618075058716
Data shape:  (140, 18, 3)
Mean torso:  274.49230143795376
Data shape:  (122, 18, 3)
Mean torso:  276.056770970454
Data shape:  (113, 18, 3)
Mean torso:  386.10674117118276
Data shape:  (117, 18, 3)
Mean torso:  388.8324699035529
Data shape:  (108, 18, 3)
Mean torso:  359.69948300318237
Data shape:  (146, 18, 3)
Mean torso:  362.94356251552193
Data shape:  (92, 18, 3)
Mean torso:  364.56482601788053
Data shape:  (120, 18, 3)
Mean torso:  364.55769028003914
Data shape:  (123, 18, 3)
Mean torso:  271.04817985102164
Data shape:  (100, 18, 3)
Mean torso:  274.544553716319
Data shape:  (127, 18, 3)
Mean torso:  276.5870125620258
Data shape:  (102, 18, 3)
Mean torso: 389.4333451516014
Data shape:  (108, 18, 3)
Mean torso:  366.72702499907354
Data shape:  (80, 18, 3)
Mean torso:  382.9545547331215


In [3]:
for video in good_videos:
    frames = video

    side = detect_perspective(frames)

    if (side == pose.Side.right):
        upper_arm_vecs = np.array([pose.Joint.vectorFromJoints(frame.rshoulder, frame.relbow) for frame in frames])
        lower_arm_vecs = np.array([pose.Joint.vectorFromJoints(frame.relbow, frame.rwrist) for frame in frames])
    else:
        upper_arm_vecs = np.array([pose.Joint.vectorFromJoints(frame.lshoulder, frame.lelbow) for frame in frames])
        lower_arm_vecs = np.array([pose.Joint.vectorFromJoints(frame.lelbow, frame.lwrist) for frame in frames])

    # print(lower_arm_vecs)
    print (upper_arm_vecs.shape)
    print (upper_arm_vecs[0])

    #Find the unit vector vec/mag(normalization axis=1)
    upper_arm_vecs = upper_arm_vecs/ np.expand_dims(np.linalg.norm(upper_arm_vecs, axis=1), axis=1)
    lower_arm_vecs = lower_arm_vecs/ np.expand_dims(np.linalg.norm(upper_arm_vecs, axis=1), axis=1)

    #Angle = cos inverse of product of two unit vectors
    # np.multiply(upper_arm_vecs, lower_arm_vecs)
    # np.sum(np.multiply(upper_arm_vecs, lower_arm_vecs), axis=1)
    upper_arm_forearm_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(upper_arm_vecs, lower_arm_vecs), axis=1), -1.0, 1.0)))
    print(upper_arm_forearm_angle)



Primary arm: Right
(114, 2)
[0.08156564 0.51118136]
[ 51.78879228  50.97291159  50.97237164  51.44469337  52.4608933
  54.08114794  68.67544868  54.01911671  53.39371522  53.35177509
  53.2255675   53.3932587   54.75829998  68.58601926  69.90420195
  70.28083126  70.44803663  70.60925159  73.35353752  73.35194827
  75.54615329  67.23848223  68.37273436  69.88907797  72.5067948
  74.31114936  76.94680804  77.95305685  78.724543    81.85425063
  84.54119758  84.5122068   88.75372466  89.39878712  92.17110695
  94.20029834  96.02639477  97.05689086  98.75878359 100.00109192
 101.21586801 102.94083629 104.08484291 104.08600459 104.11090451
 105.34486038 108.47423193 107.00435924 105.76212587 106.76936136
 107.41978279 105.84328269 106.18242565 105.75541019 107.14188185
 108.14037814 108.38502102 108.25354676 108.26266678 106.91265532
 107.57570511 107.22584314 108.54511768 107.59503125 107.24519476
 106.5926311  106.24500958 105.64910323 105.29119767 105.46295148
 105.36606756 103.89173249

In [4]:
upper_arm_vecs = upper_arm_vecs/ np.expand_dims(np.linalg.norm(upper_arm_vecs, axis=1), axis=1)
lower_arm_vecs = lower_arm_vecs/ np.expand_dims(np.linalg.norm(upper_arm_vecs, axis=1), axis=1)